## Install Dependencies

In [ ]:
!pip install langchain langchain-community langchain-core pinecone fastembed langchain-pinecone

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from google.colab import userdata
import time

## Load embedding model

In [ ]:
class BGEM3Embeddings(Embeddings):
    def __init__(self, model_name: str = "BAAI/bge-m3"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return [
            self.model.encode(text, normalize_embeddings=True).tolist()
            for text in texts
        ]

    def embed_query(self, text):
        return self.model.encode(
            text,
            normalize_embeddings=True
        ).tolist()

In [ ]:
embedding_model = BGEM3Embeddings()

## Create index in vector store

In [ ]:
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
pc_index_name = "gic-bge-m3"

if pc_index_name in existing_indexes:
  print(f"Vector store index '{pc_index_name}' found. Deleting existing...")
  pc.delete_index(pc_index_name)


print(f"Creating new vector store...")

pc.create_index(
  name=pc_index_name,
  dimension=1024,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

while not pc.describe_index(pc_index_name).status['ready']:  # Wait for the index to be ready
  time.sleep(1)
  print(f"Successfully created vector store index '{pc_index_name}'.")


Creating new vector store...


## Store contexts in vector store index

In [ ]:
import pandas as pd

data = pd.read_csv("gic_data_contexts_final_with_meta.csv")
print(len(data))

1699


In [ ]:
data.head()

,context_id,context_title,context_content,main_category_id,main_category_name,sub_category_name,sub_category_id,service_name,service_id,service_link,context_with_metadata
0,1-87-665-1,Educational Publications Department Book Sales...,"There are 6 sales outlets of books, successful...",1,Education & Training,Education Publications,87,Sales Outlets of Books,665,https://gic.gov.lk/gic/index.php/en/component/...,Service: Sales Outlets of Books\nMain Category...
1,1-87-1481-1,Publication Prices - Sinhala Publications,Publication Price Sinhala Publications Rs. Cts...,1,Education & Training,Education Publications,87,Museum Publications,1481,https://gic.gov.lk/gic/index.php/en/component/...,Service: Museum Publications\nMain Category: E...
2,1-87-1481-2,Publication Prices - English Publications,English Publications\n21. Some Sinhala Combati...,1,Education & Training,Education Publications,87,Museum Publications,1481,https://gic.gov.lk/gic/index.php/en/component/...,Service: Museum Publications\nMain Category: E...
3,1-87-1481-3,Publication Prices - English and Tamil Publica...,36. The Pleistocene of Ceylon 390 00\n37. Colo...,1,Education & Training,Education Publications,87,Museum Publications,1481,https://gic.gov.lk/gic/index.php/en/component/...,Service: Museum Publications\nMain Category: E...
4,1-87-662-1,Departmental Administration and Record Keeping,1. Maintaining the personal files of the offic...,1,Education & Training,Education Publications,87,Services of Establishment Unit of Educational ...,662,https://gic.gov.lk/gic/index.php/en/component/...,Service: Services of Establishment Unit of Edu...


In [ ]:
from langchain_core.documents import Document

docs = []

for i, row in data.iterrows():
    if i % 20 == 0:
        print(f"Adding context {i}")

    doc = Document(
        page_content=row["context_with_metadata"],
        metadata={
            "context_id": row["context_id"],
            "context_title": row["context_title"],
            "main_category_id": int(row["main_category_id"]),
            "main_category_name": row["main_category_name"],
            "sub_category_id": int(row["sub_category_id"]),
            "sub_category_name": row["sub_category_name"],
            "service_id": int(row["service_id"]),
            "service_name": row["service_name"],
            "service_link": row["service_link"],
            "language": "en"
        }
    )

    docs.append(doc)

print(len(docs))

In [ ]:
index = pc.Index(pc_index_name)
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

In [ ]:
embeddings = []
for i, doc in enumerate(docs):
    if i % 20 == 0:
        print(f"Computing embedding {i}")
    emb = embedding_model.embed_documents([doc.page_content])[0]
    embeddings.append(emb)


In [ ]:
print(len(embeddings))

1699


In [ ]:
vectors = [
    (str(i), emb, doc.metadata)
    for i, (doc, emb) in enumerate(zip(docs, embeddings))
]

In [ ]:
batch_size = 25
index = pc.Index(pc_index_name)

for i in range(0, len(vectors), batch_size):
    batch = vectors[i:i+batch_size]
    print(f"Uploading batch {i} - {i + len(batch)}")
    index.upsert(vectors=batch)

Uploading batch 0 - 25
Uploading batch 25 - 50
Uploading batch 50 - 75
Uploading batch 75 - 100
Uploading batch 100 - 125
Uploading batch 125 - 150
Uploading batch 150 - 175
Uploading batch 175 - 200
Uploading batch 200 - 225
Uploading batch 225 - 250
Uploading batch 250 - 275
Uploading batch 275 - 300
Uploading batch 300 - 325
Uploading batch 325 - 350
Uploading batch 350 - 375
Uploading batch 375 - 400
Uploading batch 400 - 425
Uploading batch 425 - 450
Uploading batch 450 - 475
Uploading batch 475 - 500
Uploading batch 500 - 525
Uploading batch 525 - 550
Uploading batch 550 - 575
Uploading batch 575 - 600
Uploading batch 600 - 625
Uploading batch 625 - 650
Uploading batch 650 - 675
Uploading batch 675 - 700
Uploading batch 700 - 725
Uploading batch 725 - 750
Uploading batch 750 - 775
Uploading batch 775 - 800
Uploading batch 800 - 825
Uploading batch 825 - 850
Uploading batch 850 - 875
Uploading batch 875 - 900
Uploading batch 900 - 925
Uploading batch 925 - 950
Uploading batch 950 

In [ ]:
stats = index.describe_index_stats()
print("Pinecone index stats:", stats)

# Add "text" metadata for embeddings

In [ ]:
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
pc_index_name = "gic-bge-m3"

In [ ]:
index = pc.Index(pc_index_name)
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

In [ ]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1699}},
 'total_vector_count': 1699,
 'vector_type': 'dense'}

In [ ]:
index.update(
    id="0",
    set_metadata={
        "text": docs[0].page_content,
    }
)

{}

In [ ]:
for i, doc in enumerate(docs):
    if i % 20 == 0:
        print(f"Updating text {i}")
    index.update(
      id=str(i),
      set_metadata={
          "text": doc.page_content,
      }
    )

Updating text 0
Updating text 20
Updating text 40
Updating text 60
Updating text 80
Updating text 100
Updating text 120
Updating text 140
Updating text 160
Updating text 180
Updating text 200
Updating text 220
Updating text 240
Updating text 260
Updating text 280
Updating text 300
Updating text 320
Updating text 340
Updating text 360
Updating text 380
Updating text 400
Updating text 420
Updating text 440
Updating text 460
Updating text 480
Updating text 500
Updating text 520
Updating text 540
Updating text 560
Updating text 580
Updating text 600
Updating text 620
Updating text 640
Updating text 660
Updating text 680
Updating text 700
Updating text 720
Updating text 740
Updating text 760
Updating text 780
Updating text 800
Updating text 820
Updating text 840
Updating text 860
Updating text 880
Updating text 900
Updating text 920
Updating text 940
Updating text 960
Updating text 980
Updating text 1000
Updating text 1020
Updating text 1040
Updating text 1060
Updating text 1080
Updating te